In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model.safetensors.index.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00001-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/config.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00003-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00002-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00007-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/README.md
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00008-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/tokenizer.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/tokenizer_config.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00005-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00006-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/special_tokens_map.json
/kaggle/input/gemma-2/transformer

In [4]:
import pandas as pd
import numpy as np
from metric import PerplexityCalculator
from collections import deque
import random
from typing import List, Tuple

In [5]:
# Charger les données
input_data = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')
input_data

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


In [6]:
# Initialiser le calculateur de perplexité
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
import numpy as np

# Initialize list to store perplexity scores
scores = []

# Iterate over rows in the dataset
for idx, row in input_data.iterrows():
    # Retrieve and calculate perplexity score for each text entry
    score = scorer.get_perplexity(row.text)
    
    # Log details for each row
    print(f"Row {idx}: {row.text}")
    print(f"Perplexity Score: {score:.2f}\n")
    
    # Append score to the list
    scores.append(score)

# Calculate and print the mean perplexity score
mean_score = np.mean(scores)
print(f"Mean Perplexity Score: {mean_score:.2f}")

Row 0: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
Perplexity Score: 3945.12

Row 1: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
Perplexity Score: 6106.05

Row 2: yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
Perplexity Score: 1113.49

Row 3: yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
Perplexity Score: 1280.80

Row 4: hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you f

In [8]:
# Extraire la première phrase
original_sentence = input_data.iloc[0]['text']
words = original_sentence.split()
n = len(words)

In [21]:
class WhaleOptimizationQAP:
    def __init__(self, n_whales: int = 20, max_iter: int = 80) -> None:
        self.words = words
        self.n = n
        self.n_whales = n_whales
        self.max_iter = max_iter
        self.cache = {}
        self.fromCache = 0
        self.attempt = 0
        self.calculted = 0

    def _compute_A(self, a: float) -> np.ndarray:
        r = np.random.uniform(0.0, 1.0, self.n)
        return (2.0 * np.multiply(a, r)) - a

    def _compute_C(self) -> np.ndarray:
        return 2.0 * np.random.uniform(0.0, 1.0, self.n)

    def __calculate_fitness(self, solution: List[int]) -> float:
        self.attempt += 1
        sol = tuple(solution)
        if sol in self.cache:
            self.fromCache += 1
            return self.cache[sol]

        permuted_sentence = ' '.join([self.words[i] for i in solution])
        cost = scorer.get_perplexity(permuted_sentence)
        self.cache[sol] = cost
        self.calculted += 1
        return cost

    def __create_initial_sols(self) -> List[int]:
        return list(np.random.permutation(self.n))

    def __local_search(self, solution: List[int]) -> Tuple[List[int], float]:
        return solution, self.__calculate_fitness(solution)

    def __generate_unique_solution(self) -> List[int]:
        """ Génère une solution unique de longueur n. """
        return list(np.random.permutation(self.n))

    def optimize_with_local_search(self) -> Tuple[List[int], float]:
        population = []
        fitness_values = []
        
        for _ in range(self.n_whales):
            solution = self.__generate_unique_solution()
            improved_solution, improved_fitness = self.__local_search(solution)
            population.append(improved_solution)
            fitness_values.append(improved_fitness)

        best_idx = np.argmin(fitness_values)
        best_pos = population[best_idx].copy()
        best_fitness = fitness_values[best_idx]

        for t in range(self.max_iter):
            for i in range(self.n_whales):
                a = 2 - t * (2 / self.max_iter)
                A = self._compute_A(a)
                C = self._compute_C()
                
                # Update position based on WAO logic
                new_pos = best_pos.copy()
                for j in range(self.n):
                    if random.random() < 0.5:
                        new_index = int(best_pos[j] + A[j])
                    else:
                        new_index = int(new_pos[j] + C[j] * (best_pos[j] - new_pos[j]))

                    # Assurer que les indices sont valides
                    new_pos[j] = np.clip(new_index, 0, self.n - 1)

                # S'assurer que new_pos est une permutation unique
                unique_indices = set(new_pos)
                if len(unique_indices) < self.n:
                    # Remplir avec des indices manquants
                    missing_indices = set(range(self.n)) - unique_indices
                    new_pos = list(unique_indices) + list(missing_indices)

                new_pos = np.random.permutation(new_pos)[:self.n]  # Créer une permutation valide

                improved_pos, improved_fitness = self.__local_search(new_pos)
                population[i] = improved_pos
                fitness_values[i] = improved_fitness
                
                if improved_fitness < best_fitness:
                    best_pos = improved_pos.copy()
                    best_fitness = improved_fitness

        return best_pos, best_fitness

In [22]:

import time
import random

In [24]:
# Boucle d'exécution
s = time.time()

costs = []
best_overall_solution = None
best_overall_cost = float('inf')

for i in range(10):
    woaqap = WhaleOptimizationQAP(n_whales=3, max_iter=10)
    _s = time.time()
    best_solution, best_cost = woaqap.optimize_with_local_search()
    _e = time.time()
    
    print(f"[{i+1}] solution: {best_solution} | cost: {best_cost} | time: {_e - _s:.4f} seconds")
    
    costs.append(best_cost)
    
    if best_cost < best_overall_cost:
        best_overall_cost = best_cost
        best_overall_solution = best_solution

e = time.time()

# Afficher les résultats finaux
print("\nBest Solution (indices):", best_overall_solution)
print("Best Cost (Perplexity):", best_overall_cost)
print("Average Cost:", sum(costs) / len(costs))
print("Total Time:", e - s, "seconds")

[1] solution: [8 2 9 5 1 0 6 7 4 3] | cost: 914.3269222248507 | time: 3.3848 seconds
[2] solution: [8 2 9 3 0 4 6 1 7 5] | cost: 1179.764977630724 | time: 3.2987 seconds
[3] solution: [8 6 9 1 3 5 4 7 0 2] | cost: 804.4576959205698 | time: 3.2938 seconds
[4] solution: [7 9 3 0 5 1 8 2 6 4] | cost: 1501.4571387578694 | time: 3.2533 seconds
[5] solution: [8 6 5 3 2 0 7 4 9 1] | cost: 891.000976860342 | time: 3.2653 seconds
[6] solution: [5 0 9 3 1 6 2 7 4 8] | cost: 1525.1947358926054 | time: 3.3330 seconds
[7] solution: [8 2 3 5 4 1 6 7 0 9] | cost: 1207.0508442542696 | time: 3.2844 seconds
[8] solution: [8 6 5 1 3 9 4 2 0 7] | cost: 848.4305033058594 | time: 3.2911 seconds
[9] solution: [8 2 1 9 6 4 3 5 0 7] | cost: 1135.0142252583694 | time: 3.3023 seconds
[10] solution: [7 9 6 5 0 3 2 8 1 4] | cost: 1186.8654615456796 | time: 3.2828 seconds

Best Solution (indices): [8 6 9 1 3 5 4 7 0 2]
Best Cost (Perplexity): 804.4576959205698
Average Cost: 1119.356348165114
Total Time: 32.99266195